# Hands-on Exercise (4)

**Reconstruction of Charged Particles (Tracking)**

# Reconstruction of Charged Particles (Tracking)

## Tracking with Annealing Technique

In this exercise, we try to find tracks from detector hits using simulated annealing technique. This is based on the <a href="https://github.com/derlin/hepqpr-qallse" target="_blank">hepqpr-qallse</a> framework developed by Lucy Linder and LBNL group.

First, let us import necessary modules.

In [1]:
# Tested with python 3.10.11, qiskit 0.42.1, numpy 1.23.5, scipy 1.9.3
import numpy as np
#import matplotlib.pyplot as plt

from qiskit_aer import AerSimulator
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator, Sampler, BackendEstimator
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit_algorithms.minimum_eigensolvers import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.gradients import ParamShiftEstimatorGradient
from qiskit_algorithms.optimizers import SPSA, COBYLA
from qiskit_optimization.applications import OptimizationApplication


Next, create a dataset of detector hits. The parameter `density` controls how many particles in an event are included in the dataset. The density = 0.0015 means the particle density corresponds to about 0.15% of a typical HL-LHC collision event.


In [7]:
import os
import logging
from hepqpr.qallse.dsmaker import create_dataset

#density = 0.0015
density = 0.001

output_path = os.getcwd()+'/ds'
prefix = 'ds'+str(density)

metadata, path = create_dataset(
    density=density,
    output_path=output_path,
    prefix=prefix,
    gen_doublets=True
)

2024-12-07T17:10:01.307 [hepqpr.qallse.dsmaker.dsmaker INFO ] Doublets (len=39) generated in f/Users/terashi/Work/QC/qallse_anneal_niigata_2024/ds/ds0.001/event000001000.


You will see that the dataset is created under ds in your working directory.

Next, QUBO is produced from the dataset by reconstructing doublets, triplets and quadruplets from the hits and checking their relative geometries.


In [8]:
from hepqpr.qallse import *

# ==== BUILD CONFIG
loglevel = logging.INFO

input_path = os.getcwd()+'/ds/'+prefix+'/event000001000-hits.csv'
output_path = os.getcwd()+'/ds/'+prefix+'/'

model_class = QallseD0  # model class to use
extra_config = dict()  # model config

dump_config = dict(
    output_path = os.getcwd()+'/ds/'+prefix+'/',
    prefix=prefix+'_',
    xplets_kwargs=dict(format='json', indent=3), # use json (vs "pickle") and indent the output
    qubo_kwargs=dict(w_marker=None, c_marker=None) # save the real coefficients VS generic placeholders
)

# ==== configure logging
logging.basicConfig(
    stream=sys.stderr,
    format="%(asctime)s.%(msecs)03d [%(name)-15s %(levelname)-5s] %(message)s",
    datefmt='%Y-%m-%dT%H:%M:%S')

logging.getLogger('hepqpr').setLevel(loglevel)

# ==== build model
# load data
dw = DataWrapper.from_path(input_path)
doublets = pd.read_csv(input_path.replace('-hits.csv', '-doublets.csv'))

# build model
model = model_class(dw, **extra_config)
model.build_model(doublets)

# dump model to a file
dumper.dump_model(model, **dump_config)

2024-12-07T17:10:03.747 [hepqpr.qallse.qallse_d0 INFO ] created 29 doublets.
2024-12-07T17:10:03.749 [hepqpr.qallse.qallse_d0 INFO ] created 16 triplets.
2024-12-07T17:10:03.750 [hepqpr.qallse.qallse_d0 INFO ] created 15 quadruplets.
2024-12-07T17:10:03.752 [hepqpr.qallse.qallse_d0 INFO ] Model built in 0.01s. doublets: 29/0, triplets: 16/0, quadruplets: 15
2024-12-07T17:10:03.753 [hepqpr.qallse.qallse_d0 INFO ] MaxPath done in 0.00s. doublets: 12, triplets: 15, quadruplets: 15 (dropped 0)
2024-12-07T17:10:03.888 [hepqpr.qallse.qallse_d0 INFO ] Qubo generated in 0.00s. Size: 75. Vars: 15, excl. couplers: 45, incl. couplers: 15


{('83840_90726_96872', '83840_90726_96872'): 0.1453120217518618,
 ('38384_44948_76407', '38384_44948_76407'): 0.17971795504280036,
 ('22838_31225_38384', '22838_31225_38384'): 0.04647295276079133,
 ('38384_44948_83840', '38384_44948_83840'): 0.17311859128234375,
 ('31225_38384_76407', '31225_38384_76407'): 0.02410271326415958,
 ('76407_83840_96872', '76407_83840_96872'): 0.1407708018806541,
 ('31225_44948_76407', '31225_44948_76407'): 0.06140279743056504,
 ('38384_76407_83840', '38384_76407_83840'): 0.16080369188713034,
 ('22838_31225_44948', '22838_31225_44948'): 0.03265154080105226,
 ('31225_38384_44948', '31225_38384_44948'): 0.07888777889940413,
 ('44948_76407_90726', '44948_76407_90726'): 0.1544495972447824,
 ('76407_83840_90726', '76407_83840_90726'): 0.19806956574975726,
 ('44948_83840_90726', '44948_83840_90726'): 0.17063283788610467,
 ('76407_90726_96872', '76407_90726_96872'): 0.020402169246270974,
 ('44948_76407_83840', '44948_76407_83840'): 0.1417855528334382,
 ('31225_3838

Set up the annealing job by loading QUBO.

In [9]:
import pickle
from os.path import join as path_join

from hepqpr.qallse.other.stdout_redirect import capture_stdout
from hepqpr.qallse.other.dw_timing_recorder import solver_with_timing, TimingRecord
from hepqpr.qallse.plotting import *


# ==== RUN CONFIG
nreads = 10
nseed = 1000000

loglevel = logging.INFO

input_path = os.getcwd()+'/ds/'+prefix+'/event000001000-hits.csv'
qubo_path = os.getcwd()+'/ds/'+prefix+'/'

# ==== configure logging
logging.basicConfig(
    stream=sys.stdout,
    format="%(asctime)s.%(msecs)03d [%(name)-15s %(levelname)-5s] %(message)s",
    datefmt='%Y-%m-%dT%H:%M:%S')

logging.getLogger('hepqpr').setLevel(loglevel)

# ==== build model
# load data
dw = DataWrapper.from_path(input_path)
pickle_file = prefix+'_qubo.pickle'
with open(path_join(qubo_path, pickle_file), 'rb') as f:
    Q = pickle.load(f)
#print(Q)

import time
start_time = time.process_time()


Perform simulated annealing using neal software.

If it's successfully done, you can see a file "plot_ds..._tracks_found.html" in created in your working directory. This plot displays the detector hits in QUBO, projected onto a plane perpendicular to the beam axis, and shows which detector hits are successfully selected to from reconstructed tracks. The green lines correspond to reconstructed tracks, the blue lines missing tracks (i.e, tracks that were not reconstructed) and the red ones fake tracks (i.e, misreconstructed tracks that do not mach truth particles).

In [10]:
# Sample qubo

# --- neal
import neal
sampler = neal.SimulatedAnnealingSampler()
#import dimod
#sampler = dimod.RandomSampler()
response = sampler.sample_qubo(Q, num_reads=nreads, seed=nseed)

exec_time = time.process_time() - start_time
print(f'QUBO of size {len(Q)} sampled in {exec_time:.2f}s (NEAL).')
print('')


# get the results
all_doublets = Qallse.process_sample(next(response.samples()))
final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

# compute stats
en0 = dw.compute_energy(Q)
en = response.record.energy[0]
occs = response.record.num_occurrences

p, r, ms = dw.compute_score(final_doublets)
trackml_score = dw.compute_trackml_score(final_tracks)

# print stats
print(f'SAMPLE -- energy: {en:.4f}, ideal: {en0:.4f} (diff: {en-en0:.6f})')
print(f'          best sample occurrence: {occs[0]}/{occs.sum()}')

print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(ms)}')
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

# plotting examples
dims = ['x', 'y']
dout = 'plot_'+prefix+'_tracks_found.html'
iplot_results(dw, final_doublets, ms, dims=dims, filename=dout)
#iplot_results_tracks(dw, final_tracks)

/var/folders/wg/l7y2frtd5m7f_2r2sh2lmj800000gn/T/ipykernel_59881/1750332290.py:16: DeprecationWarning:

SampleSet.samples() will return an iterable not an iterator in the future

2024-12-07T17:10:11.023 [hepqpr.qallse.track_recreater INFO ] Found 0 conflicting doublets


QUBO of size 75 sampled in 0.04s (NEAL).

SAMPLE -- energy: -3.4086, ideal: -3.4086 (diff: 0.000000)
          best sample occurrence: 1/10
SCORE  -- precision (%): 100.0, recall (%): 100.0, missing: 0
          tracks found: 1, trackml score (%): 99.99999999999999


### Attention
**Below we try to reconstruct tracks with VQE in a quantum circuit model. Since each segment is assigned to a qubit in this (naive) approach, the memory consumption quickly becomes explosive with the number of segments and the kernel will crash if the number of segments is larger than about 30 or so. Before proceeding, please make sure that the number of segments has to be less than 20-25.**

## Hamiltonian Formulation and VQE Implementation

In order to use VQE for optimization, the problem will need to be formulated in the form of Hamiltonian. If the problem is formulated such that the solution corresponds to the lowest energy state of the Hamiltonian, the VQE could solve the problem by finding such state.

### QUBO Format

Under this setup, the next step is whether a given segment is adopted as part of particle tracks or rejected as fake. In a sample of $N$ segments, the adoptation or rejection of $i$-th segment is associated to 1 or 0 of a binary variable $T_i$, and the variable $T_i$ is determined such that the objective function defined as

$$
O(b, T) = \sum_{i=1}^N a_{i} T_i + \sum_{i=1}^N \sum_{j<i}^N b_{ij} T_i T_j
$$

is minimized. Here $a_i$ is the score of $i$-th segment and $b_{ij}$ is the score of the pair of $i$- and $j$-th segments. The objective function becomes smaller by selecting segments that have smaller $a_i$ values (pointing towards the detector center) and are paired with other segments with smaller $b_{ij}$ values (more consistent with a real track) and rejecting otherwise. Once the correct segments are identified, the corresponding tracks can be reconstructed with high efficiency. Therefore, solving this minimization problem is the key to tracking.

Let us first extract the scores $a_i$ and $b_{ij}$ from the QUBO produced above (corresponding to the variable Q).


In [11]:
n_max = 100

nvar = 0
key_i = []
a_score = np.zeros(n_max)
for (k1, k2), v in Q.items():
    if k1 == k2:
        a_score[nvar] = v
        key_i.append(k1)
        nvar += 1
a_score = a_score[:nvar]

b_score = np.zeros((n_max,n_max))
for (k1, k2), v in Q.items():
    if k1 != k2:
        for i in range(nvar):
            for j in range(nvar):
                if k1 == key_i[i] and k2 == key_i[j]:
                    if i < j:
                        b_score[j][i] = v
                    else:
                        b_score[i][j] = v

b_score = b_score[:nvar,:nvar]

print(f'# of Segments: {nvar}')
# Print out the first 5x5
print(a_score[:5])
print(b_score[:5, :5])

# of Segments: 15
[0.14531202 0.17971796 0.04647295 0.17311859 0.02410271]
[[ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.        ]
 [ 0.          1.         -0.20368915  1.          0.        ]]


### Ising Format

The QUBO objective function is not the form of Hamiltonian (i.e, not Hermitian operator). Therefore, the objective function needs to be transformed before solving with VQE. Given that $T_i$ takes a binary value $\{0, 1\}$, a new variable $s_i$ with values of $\{+1, -1\}$ can be defined by

$$
T_i = \frac{1}{2} (1 - s_i).
$$

Note that $\{+1, -1\}$ is the eigenvalue of Pauli operator. By replacing $s_i$ with Pauli $Z$ operator acting on $i$-th qubit, we can obtain the following Hamiltonian for which the computational basis states in $N$-qubit system correspond to the eigenstates that encode adoptation or rejection of the segments:

$$
H(h, J, s) = \sum_{i=1}^N h_i Z_i + \sum_{i=1}^N \sum_{j<i}^N J_{ij} Z_i Z_j + \text{(constant)}
$$

The form of this Hamiltonian is the same as Ising model Hamiltonian, which often appears in various fields of natural science. The $\text{constant}$ is a constant term and has no impact in variational method, hence ignored in the rest of this exercise.

### Exercise

By following the above prescription, please calculate the coefficients $h_i$ and $J_{ij}$ of the Hamiltonian in the next cell.

In [12]:
num_qubits = nvar

coeff_h = np.zeros(num_qubits)
coeff_J = np.zeros((num_qubits, num_qubits))

# Calculate coeff_h and coeff_J from a_score and b_score
coeff_h = -(a_score / 2. + (np.sum(b_score, axis=0) + np.sum(b_score, axis=1)) / 4.)
coeff_J = b_score / 4.

Next, let us define the Hamiltonian used in VQE as a SparsePauliOp object. In VQE exercise of Hands-on Exercise (3), the SparsePauliOp was used to define a single Pauli string $ZXY$, but the same class can be used for the sum of Pauli strings. For example,

$$
H = 0.2 IIZ + 0.3 ZZI + 0.1 ZIZ
$$

can be expressed as

```python
H = SparsePauliOp(['IIZ', 'ZZI', 'ZIZ'], coeffs=[0.2, 0.3, 0.1])
```

Note that the qubits are ordered from right to left (the most right operator acts on the 0-th qubit) according to the rule in Qiskit.

### Exercise

Pick up all the Pauli strings with non-zero coefficients and make the array of corresponding coefficients.

In [13]:
pauli_products = []
coeffs = []

for iq in range(num_qubits):
    if np.isclose(coeff_h[iq], 0.):
        continue

    pauli_products.append(('I' * (num_qubits - iq - 1)) + 'Z' + ('I' * iq))
    coeffs.append(coeff_h[iq])

for iq in range(num_qubits):
    for jq in range(iq):
        if np.isclose(coeff_J[iq, jq], 0.):
            continue

        pauli = 'I' * (num_qubits - iq - 1)
        pauli += 'Z'
        pauli += 'I' * (iq - jq - 1)
        pauli += 'Z'
        pauli += 'I' * jq
        pauli_products.append(pauli)

        coeffs.append(coeff_J[iq, jq])


hamiltonian = SparsePauliOp(pauli_products, coeffs=coeffs)

### Executing VQE

Now we try to approximately obtain the lowest energy eigenvalues using VQE with the Hamiltonian defined above. But, before doing that, let us diagonalize the Hamiltonian matrix and calculate the exact energy eigenvalues and eigenstates.

In [14]:
# Diagonalize the Hamiltonian and calculate the energy eigenvalues and eigenstates
ee = NumPyMinimumEigensolver()
result_diag = ee.compute_minimum_eigenvalue(hamiltonian)

# Print out the combination of qubits corresponding to the lowest energy
print(f'Minimum eigenvalue (diagonalization): {result_diag.eigenvalue.real}')
# Expand the state with computational bases and select the one with the highest probability
optimal_segments_diag = OptimizationApplication.sample_most_likely(result_diag.eigenstate)
print(f'Optimal segments (diagonalization): {optimal_segments_diag}')

Minimum eigenvalue (diagonalization): -13.89232589651172
Optimal segments (diagonalization): [1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.]


Now we move to VQE.

In [15]:
backend = AerSimulator()
# Create Estimator instance
estimator = BackendEstimator(backend)

# Define variational form of VQE using a built-in function called TwoLocal.
ansatz = TwoLocal(num_qubits, 'ry', 'cz', 'linear', reps=1)

# Optimizer
optimizer_name = 'SPSA'

if optimizer_name == 'SPSA':
    optimizer = SPSA(maxiter=300)
    grad = ParamShiftEstimatorGradient(estimator)

elif optimizer_name == 'COBYLA':
    optimizer = COBYLA(maxiter=500)
    grad = None

# Initialize parameters with random values
rng = np.random.default_rng()
init = rng.uniform(0., 2. * np.pi, size=len(ansatz.parameters))

/var/folders/wg/l7y2frtd5m7f_2r2sh2lmj800000gn/T/ipykernel_59881/2402505469.py:3: DeprecationWarning:

The class ``qiskit.primitives.backend_estimator.BackendEstimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `BackendEstimator` class is `BackendEstimatorV2`.



In [16]:
# Make VQE object and search for the ground state
vqe = VQE(estimator, ansatz, optimizer, gradient=grad, initial_point=init)
result_vqe = vqe.compute_minimum_eigenvalue(hamiltonian)

# Create state vector from the ansatz using optimized parameters
optimal_state = Statevector(ansatz.assign_parameters(result_vqe.optimal_parameters))

# Print out the combination of qubits with the lowest energy
print(f'Minimum eigenvalue (VQE): {result_vqe.eigenvalue.real}')
optimal_segments_vqe = OptimizationApplication.sample_most_likely(optimal_state)
print(f'Optimal segments (VQE): {optimal_segments_vqe}')

Minimum eigenvalue (VQE): -13.014322899199367
Optimal segments (VQE): [1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.]


Check again reconstructed tracks in the detector plane for fun. You can switch the results of exact diagonalization and VQE with the variable `type`.

In [17]:
from hepqpr.qallse import DataWrapper, Qallse, TrackRecreaterD
from hepqpr.qallse.plotting import iplot_results, iplot_results_tracks
from hepqpr.qallse.utils import diff_rows

# Results tp show: diag = Exact diagonalization, vqe = VQE
type = "diag"
#type = "vqe"

if type == "diag":
    optimal_segments = optimal_segments_diag
elif type == "vqe":
    optimal_segments = optimal_segments_vqe

samples = dict(zip(key_i, optimal_segments))

# get the results
all_doublets = Qallse.process_sample(samples)

final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

#dw = DataWrapper.from_path('data/event000001000-hits.csv')
input_path = os.getcwd()+'/ds/'+prefix+'/event000001000-hits.csv'
dw = DataWrapper.from_path(input_path)

p, r, ms = dw.compute_score(final_doublets)
trackml_score = dw.compute_trackml_score(final_tracks)

print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(ms)}')
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

dims = ['x', 'y']
_, missings, _ = diff_rows(final_doublets, dw.get_real_doublets())
dout = 'plot-ising_'+type+'_found_tracks.html'
iplot_results(dw, final_doublets, missings, dims=dims, filename=dout)

2024-12-07T17:12:10.650 [hepqpr.qallse.track_recreater INFO ] Found 0 conflicting doublets


SCORE  -- precision (%): 100.0, recall (%): 100.0, missing: 0
          tracks found: 1, trackml score (%): 99.99999999999999
